<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas Lab

## Input

### Import libraries

In [ ]:
import pickle
import os
from unidecode import unidecode
from difflib import SequenceMatcher
import re

## Model

### Pickel functions

#### Pickle dump

In [ ]:
def pdump(
    output_dir,
    object_to_dump,
    file_to_dump_to,
):
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, f'{file_to_dump_to}.pickle')
    with open(file_path, 'wb') as file:
        pickle.dump(object_to_dump, file)

#### Pickle load

In [ ]:
def pload(
    output_dir,
    file_to_load_from
):
    file_path = os.path.join(output_dir, f'{file_to_load_from}.pickle')
    try:
        with open(file_path, 'rb') as file:
            return pickle.load(file)
    except:
        return None

### Sequence Matcher from df

In [ ]:
def remove_accent(string):
    # Use the unidecode function to remove accents
    string_without_accent = unidecode(string)
    return string_without_accent.lower()

def are_identical(string1, string2):
    string1 = remove_accent(string1)
    string2 = remove_accent(string2)
    # Create a SequenceMatcher object
    matcher = SequenceMatcher(None, string1, string2)
    
    # Get the ratio of similarity between the two strings
    similarity_ratio = matcher.ratio()
    # If the ratio is 1.0, the strings are identical
    if similarity_ratio > 0.9:
        return True
    else:
        return False
    
def find_crm_match(
    df,
    col_crm,
    value
):
    crm = False
    for x in df[col_crm].unique():
        if are_identical(value, str(x)):
            crm = True
            break
    return crm

### Remove emojis

In [ ]:
def remove_emojis(text):
    # Emoji pattern
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text
    text = emoji_pattern.sub(r'', text)
    return text.strip()

### Format value

In [ ]:
def format_number(num):
    num = str("{:,.0f}".format(abs(num))).replace(",", " ")
    return num

## Output